In [1]:
%gui asyncio

In [2]:
import ocs_academic_hub
from ocs_academic_hub import OMFClient
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
import pandas as pd
import requests
import asyncio
import io
import json
import os
from dateutil import parser
from dateutil.tz import *
import datetime
import chardet

In [3]:
# ocs_academic_hub.version()  # version >= 0.67.0 

# OMF Client to upload Pandas datafrome (df) with time-series data
#
# api_key is distribute by OSIsoft, acts both as an API key (Azure API Mgmt) and for data routing
#
# Code:
#    omf_client = OMFClient(api_key, asset_name)
#    omf_client.update_tags(df)
# 
# where each named column in df becomes a tag with the following format:
#
#    <university>.<asset_name>.<column_name>

<img src="https://academichub.blob.core.windows.net/images/logo_rgb.png" align="left" alt="drawing" width="55"/>

## &nbsp;&nbsp; OSIsoft Academic Hub 

###  **CSV Load/Extract/Transfer version 2.9.2**
---
### 1) Select timestamp format
* **ISO8601: standard format for hub data request** (e.g. 2020-02-25T15:24:32.43, for more [info](https://academic.osisoft.com/time-help))
* **Unix: for column is a Unix timestamp (supports decimal digits)***
* **H00: first two columns are Date and Time**


### 2) Enter API key provided by OSIsoft
### 3) Enter experiment or asset name for data
### 4) Click `Upload` button, then select CSV file to transfer 

If CSV is in a valid format, progress information will show up until completion. 

**Note: this web application can transfer one CSV at a time. Reload this web page to restart and upload a new CSV or after an error**

In [4]:
def wait_for_change(widget, value):
    future = asyncio.Future()

    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)

    widget.observe(getvalue, value)
    return future

In [5]:
upload = widgets.FileUpload()
timestamp_format = widgets.RadioButtons(
    options=["ISO8601", "Unix", "H00"], description="Timestamp:", disabled=False
)
api_key = widgets.Text(
    value="",
    placeholder="Type provided API key",
    description="API key:",
    disabled=False,
)
experiment = widgets.Text(
    value="",
    placeholder="Name of experiment or asset",
    description="Experiment:",
    disabled=False,
)
out = Output()


def my_append_stdout(message, end="\n"):
    out.append_stdout(message + end)


def load_extract_transfer(value, api_key, timestamp_format, timezone="-07:00"):
    asset = experiment.value
    if len(asset) == 0:
        out.append_stdout("@@ experiment/asset name should not be empty\n")
        return "FAILED"

    omf_client = OMFClient(api_key, my_append_stdout)
    if not omf_client.is_ok():
        out.append_stdout(
            "\n@@ please correct reported issue (code 400 = bad API key)\n"
        )
        return "FAILED"

    try:
        file_key = list(value.keys())[0]
        detected = chardet.detect(value[file_key]["content"])
        out.append_stdout(f">> Filename: {file_key} + ({detected})\n")
        data = value[file_key]["content"].decode(detected["encoding"])
        df = pd.read_csv(io.StringIO(data))
        out.append_stdout("[df.read_csv ok]\n")

        if timestamp_format == "H00" or timestamp_format == "Unix":
            if timestamp_format == "H00":
                if list(df.columns[:2]) != ["Date", "Time"]:
                    out.append_stdout(
                        f"@@@ first two columns must be Date and Time for H00 format"
                    )
                    return "FAILED"
            elif list(df.columns[:1]) != ["Time"]:
                out.append_stdout(
                    f"@@@ first column must be called Time and contains Unix timestamps"
                )
                return "FAILED"

            if df.isnull().values.any():
                my_append_stdout(
                    f"@@@ some values are missing, please correct and resubmit"
                )
                return "FAILED"

            message = (
                "Date+Time columns" if timestamp_format == "H00" else "Time column"
            )
            out.append_stdout(f"[transforming {message} into ISO8601 timestamp]\n")
            df["Timestamp"] = parser.parse("2020-01-01T00:00Z")
            timestamp_column_index = len(df.columns) - 1
            if timestamp_format == "H00":
                for i, r in enumerate(df.itertuples()):
                    row_dict = r._asdict()
                    #
                    # Time format is HH:MM:SS:ss, needs to be HH:MM:SS.ss
                    #
                    last_colon = r.Time.rfind(":")
                    new_time = r.Time[:last_colon] + "." + r.Time[last_colon + 1 :]
                    df.iloc[i, timestamp_column_index] = parser.parse(
                        r.Date + " " + new_time + " " + timezone
                    )
                    if (i + 1) % 20 == 0:
                        out.append_stdout(".")
                        if (i + 1) % (80 * 20) == 0:
                            out.append_stdout(f" [row {i+1}]\n")
            else:
                # Unix timestamp
                df["Timestamp"] = df["Time"].apply(
                    lambda ts: datetime.datetime.fromtimestamp(ts)
                )

            new_columns = ["Timestamp"] + list(df.columns)[
                2 if timestamp_format == "H00" else 1 : -1
            ]
            out.append_stdout(f"\n[transformation done]\n")
            df = df[new_columns]
        else:
            if df.columns[0] != "Timestamp":
                out.append_stdout(
                    f"@@@ First column should be Timestamp (found {df.columns[0]})\n"
                )
                return "FAILED"

        omf_client.update_tags(
            df,
            asset,
            printg=my_append_stdout,
            info_only=True if asset == "debug" else False,
            debug=False,
        )

    except Exception as e:
        out.append_stdout(
            f"\n\n!!!! Error processing CSV, exception={e}, contact hubsupport@osisoft.com\n"
        )
        return f"@@Error {file_key}\n\n"

    return f"OK {file_key}"


async def f():
    out.append_stdout("Click *Upload* button to select CSV file for transfer to Hub \n")
    x = await wait_for_change(upload, "value")
    out.append_stdout(
        "working on load, extract and tranfer... "
        + str(list(upload.value.keys())[0])
        + "\n"
    )
    status = load_extract_transfer(
        upload.value, api_key=api_key.value, timestamp_format=timestamp_format.value
    )
    out.append_stdout(f"Upload status {status}\n")


asyncio.ensure_future(f())

display(timestamp_format)
display(api_key)
display(experiment)
display(upload)
display(out)

RadioButtons(description='Timestamp:', options=('ISO8601', 'Unix', 'H00'), value='ISO8601')

Text(value='', description='API key:', placeholder='Type provided API key')

Text(value='', description='Experiment:', placeholder='Name of experiment or asset')

FileUpload(value={}, description='Upload')

Output()